In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import lightgbm as lgb
from prophet import Prophet

from sklearn.metrics import mean_squared_error, f1_score, recall_score, precision_score, mean_absolute_error

# ML4 - временные ряды

**Задание:**
Вам предоставлены данные о кол-ве посетителей одного из крупнейших бизнес центров в Америке. Данные собирались при помощи камер-счётчиков, которые стоят на входе в бц, и фиксировали кол-во посетителей, которые входили и выходили из бц. Данные предоставлены за 15 недель с 24/07/2005 по 05/11/2005. Также имеется информация о мероприятиях в данном бц, которые проходили в это время.


**Описание данных:**

- Файл counters.csv - Имеет поля dt – дата и время измерения (данные собирались каждый полчаса, то есть за 1 час имеется 2 измерения), cnt – кол-во посетителей.
- Файл events.csv – Имеет поля dt_start – дата и время начала события, dt_end –дата и время окончания события, event_type – тип события (здесь только тип события = event для всех записей)

Данные разбиты на train (24/07/2005 - 05/10/2005) и test (06/10/2005 – 05/11/2005)

**Задание:**
1. Для каждой записи (дата+время) определить, было ли в этот момент какое-то мероприятие или нет (считать, что данные о количестве пользователей на дата+время известны)
2. Реализовать прогноз кол-ва пользователей на тестовой выборке (считать, что информация о предстоящих мероприятиях в будущем (данные из таблицы events) нам известна)
3. Реализовать класс, в котором будут методы прогнозирования, метод определения мероприятия и все им необходимые методы для препроцессинга, построения фичей
4. Ответить на вопросы: Оцените качество решения задач? Какие метрики использовали и почему?



---

## Решение



### Данные и EDA

In [6]:
# Загрузка данных
counters_train = pd.read_csv('counters_train.csv', parse_dates=['dt'])
counters_test = pd.read_csv('counters_test.csv', parse_dates=['dt'])
events_train = pd.read_csv('events_train.csv', parse_dates=['dt_start', 'dt_end'])
events_test = pd.read_csv('events_test.csv', parse_dates=['dt_start', 'dt_end'])

In [7]:
counters_train.head(5)

,dt,cnt
0,2005-07-24 00:00:00,0
1,2005-07-24 00:30:00,0
2,2005-07-24 01:00:00,0
3,2005-07-24 01:30:00,0
4,2005-07-24 02:00:00,0


In [8]:
events_train.head(5)

,dt_start,dt_end,event_type
0,2005-07-26 11:00:00,2005-07-26 14:00:00,event
1,2005-07-29 08:00:00,2005-07-29 11:00:00,event
2,2005-08-02 15:30:00,2005-08-02 16:30:00,event
3,2005-08-04 16:30:00,2005-08-04 17:30:00,event
4,2005-08-05 08:00:00,2005-08-05 11:00:00,event


In [10]:
print(counters_train.shape[0])
print(counters_test.shape[0])
print(events_train.shape[0])
print(events_test.shape[0])

3552
1488
22
8


In [12]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (20, 8)
plt.rcParams['font.size'] = 14
fig = go.Figure()
fig.add_trace(go.Scatter(x=counters_train.dt, y=counters_train.cnt, mode='lines+markers', name='Train'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=counters_test.cnt, mode='lines+markers', name='Test'))
fig.update_layout(
    title  = 'Количество поситетелей',
    xaxis_title="Дата",
    yaxis_title="Счетчики",
    legend_title="Разделение:",
    width=1500,
    height=450,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1,
        xanchor="left",
        x=0.001
    )
)
fig.show()

### Модель

Реализуем класс с методами прогнозирования, в т.ч. методы препроцессинга и построения фичей. Определим мероприятия.

Для решения задачи 1 используем бустинг. Для второй Prophet из лекции.

In [13]:
class EventPredictor:
    def __init__(self):
        self.event_model = None
        self.cnt_model = None

    def get_full_data(self, users_data, events_data):
        """Функция для получения общего набора данных"""
        data = users_data.copy()
        events = np.zeros(len(data))
        for i in range(len(events_data)):
            events += data.dt.between(events_data.iloc[i].dt_start, events_data.iloc[i].dt_end, inclusive = 'both').values
        events[events > 0] = 1
        data['event'] = events
        return data

    def preprocess_data(self, users_data, events_data, type_pred):
        if type_pred == 'event':
            data = self.get_full_data(users_data, events_data)
            X = data.drop('event', axis=1)
            y = data['event']
            X['day_of_week'] = X.dt.dt.day_of_week + 1
            X['hour_day'] = X.dt.dt.hour + 1
            X['minutes'] = X.dt.dt.minute
            X = X.drop('dt', axis=1)

            # Добавим лаговою фичу в train
            X['event_lag_3'] = y.shift(periods=3, fill_value=0)
            return X, y
        
        elif type_pred == 'counters':
            df = users_data.copy()
            df.columns = ['ds', 'y']
            holidays = events_data.copy()[['event_type', 'dt_start']]
            holidays.columns = ['holiday', 'ds']
            holidays['lower_window'] = 0
            holidays['upper_window'] = 4
            return df, holidays

    def fit_event_model(self, users_data, events_data):
        X, y = self.preprocess_data(users_data, events_data, type_pred='event')
        self.event_model = lgb.LGBMClassifier(verbose=-1)
        self.event_model.fit(X, y)
    
    def fit_cnt_model(self, users_data, events_data):
        train_df, holidays = self.preprocess_data(users_data, events_data, type_pred='counters')
        self.cnt_model = Prophet(seasonality_mode='multiplicative', holidays=holidays)
        self.cnt_model.fit(train_df)

    def predict_event(self, users_data):
        X = users_data.copy()
        X['day_of_week'] = X.dt.dt.day_of_week + 1
        X['hour_day'] = X.dt.dt.hour + 1
        X['minutes'] = X.dt.dt.minute
        X = X.drop('dt', axis=1)

        # Создадим генерацию лаговых фич таргета для теста и будем обновлять лаговые фичи предсказаниями модели на предыдущих шагах
        n = len(X)
        X['event_lag_3'] = np.zeros(n)
        y_pred = np.zeros(n)
        X.loc[0:3, 'event_lag_3'] = 0
        for i in range(n - 3):
            y_pred[i] = self.event_model.predict(np.array([X.iloc[i].values]))
            X.loc[i+3, 'event_lag_3'] = y_pred[i]
        
        return y_pred

    def predict_counters(self, time_data):
        future = time_data.copy()
        future.columns = ['ds']
        y_pred = self.cnt_model.predict(future).yhat

        # Уберем отрицательные значения у предсказаний
        y_pred[y_pred < 0] = 0
        # Округлим значения до целого числа
        y_pred = y_pred.round(0)
        return y_pred
        
    def evaluate_model(self, y_test, y_pred, type_pred):
        if type_pred == 'event':
            print(f'Recall: {recall_score(y_test,y_pred).round(4)}')
            print(f'Percision: {precision_score(y_test,y_pred).round(4)}')
            print(f'F1 score: {f1_score(y_test,y_pred).round(4)}')

        elif type_pred == 'counters':
            print(f'MAE: {np.sqrt(mean_absolute_error(y_test, y_pred)).round(4)}')
            print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)).round(4)}')


Для первой хадачи решается задача бинарной классфикации для определения было ли мероприятие. Также нам известны данные о количестве посетителей. Дополнительные фичи - день недели, час, количество минут и трейтий лаг (1.5 часа). Для второй задачи включается мультипликативная сезонность (она увеличивается со временем) и праздники.

In [14]:
ep = EventPredictor()

Обучим модели

In [15]:
ep.fit_event_model(counters_train, events_train)
ep.fit_cnt_model(counters_train, pd.concat([events_train, events_test]))

13:13:15 - cmdstanpy - INFO - Chain [1] start processing
13:13:15 - cmdstanpy - INFO - Chain [1] done processing


Получим предсказания:

In [16]:
# Прогноз для тестовой выборки
# В модель для мероприятий добавляем только информацию о времени и колчиестве посещений
# Для предсказания посетителей только время, тк события были переданы в обучении
events_pred = ep.predict_event(counters_test)
cnt_pred = ep.predict_counters(counters_test[['dt']])
cnt_pred

C:\Users\pavle\AppData\Local\Temp\ipykernel_14220\2869879771.py:62: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

C:\Users\pavle\AppData\Local\Temp\ipykernel_14220\2869879771.py:62: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

C:\Users\pavle\AppData\Local\Temp\ipykernel_14220\2869879771.py:62: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

C:\Users\pavle\AppData\Local\Temp\ipykernel_14220\2869879771.py:62: DeprecationWarning:

Conversion of an array with ndim > 0 t

0       10.0
1        9.0
2        7.0
3        4.0
4        3.0
        ... 
1483     0.0
1484     0.0
1485     0.0
1486     0.0
1487     0.0
Name: yhat, Length: 1488, dtype: float64

Посмотрим на предсказания:

In [18]:
full_train = ep.get_full_data(counters_train, events_train, )
full_test = ep.get_full_data(counters_test, events_test)

fig = go.Figure()
fig.add_trace(go.Scatter(x=counters_train.dt, y=full_train.event, mode='markers', name='Train'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=full_test.event, mode='lines+markers', name='Test'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=events_pred, mode='markers', name='Predictions'))
fig.update_layout(
    title  = 'Предсказание событий',
    xaxis_title="Дата",
    yaxis_title="Было ли событие",
    legend_title="Разделение:",
    width=1500,
    height=600,
    )
fig.show()

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=counters_train.dt, y=full_train.cnt, mode='markers', name='Train'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=full_test.cnt, mode='markers', name='Test'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=cnt_pred, mode='markers', name='Predictions'))
fig.update_layout(
    title  = 'Предсказание количества посетителей',
    xaxis_title="Дата",
    yaxis_title="Счетчик",
    legend_title="Разделение:",
    width=1500,
    height=600,
    )
fig.show()

Визуализируем вместе два результата:

In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=counters_train.dt, y=counters_train.cnt, mode='markers', name='Train_counters'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=counters_test.cnt, mode='markers', name='Test_counters'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=cnt_pred, mode='markers', name='Predictions_counters'))

fig.add_trace(go.Scatter(x=counters_train.dt, y=full_train.event*300, mode='lines+markers', name='Train_event'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=full_test.event*300, mode='lines+markers', name='Test_event'))
fig.add_trace(go.Scatter(x=counters_test.dt, y=events_pred*300, mode='lines+markers', name='Predictions_event'))
fig.update_layout(
    title  = 'Предсказание количества посетителей и мероприятия',
    xaxis_title="Дата",
    yaxis_title="Счетчики/события",
    legend_title="Разделение:",
    width=1500,
    height=600,
    )
fig.show()

### Метрики качества

Для оценки наличия мероприятия:

In [21]:
ep.evaluate_model(full_test.event, events_pred, type_pred='event')


Recall: 0.2391
Percision: 0.1964
F1 score: 0.2157


Используются классические метрики для бинарной классификации, более того событий в датасете мало, сильный дисбаланс классов.

Для оценки количества посетителей:

In [22]:
ep.evaluate_model(full_test.cnt, cnt_pred, type_pred='counters')

MAE: 4.0301
RMSE: 26.6379


Для этой задачи в качестве метрик качества выбирались метрики регрессии - MAE, RMSE. 

## Вывод

- Качество решения задачи прогнозирования было ли мероприятие можно оценить как не очень высокое. Для первой модели достаточно низкий f1 score, но учитывая дисбаланс классов и небольшое количество фичей, это значение может быть приемлемым. Также, есть проблема, что когда наблюдалось большое количество посетителей, тогда и было мероприятие, поэтому модель делала ошибки.

- Качество решения задачи определения количества посетителей можно интерпретировать так, что модель будет ошибаться примерно на 4 человека (MAE - средняя абсолютная ошибка, показывающая среднее абсолютное отклонение прогнозируемых значений от фактических), RMSE - среднее квадратичное отклонение прогнозируемых значений от фактических, которая более чувствительна к большим ошибкам, чем MAE.
